In [1]:
#Libs
import pandas as pd

In [4]:
def getSampleData():
  return pd.read_csv("/content/drive/MyDrive/Sample getTransactions API data.csv")

In [7]:
raw = getSampleData()
raw.head(30)

,type,id,status,description,amount,account,balance,direction,class,institution,connection,enrich,transactionDate,postDate,subClass,links
0,transaction,4957bcf6-f18f-43d7-94df-71f44a0bcf32,posted,Wdl ATM WES IGA BALGOWLAH HGT BALGOWL AU,-200.0,d3de1ca1,22109.56,debit,cash-withdrawal,AU00000,1bd6c08e-8457-4a74-bdd9-514df489d27e,NaN,3/08/2023,2023-08-03T00:07:36,NaN,"{""account"":""https://au-api.basiq.io/users/6a52..."
1,transaction,0d6e11ab-e28e-4de0-a152-600cc44fb61c,posted,Non Hooli ATM Withdrawal Fee,-2.5,070c1d68-0ee0-477a-9679-294ea7059939,22309.56,debit,bank-fee,AU00000,1bd6c08e-8457-4a74-bdd9-514df489d27e,NaN,3/08/2023,2023-08-03T00:00:00Z,"{\title\"":\""\""","\""code\"":\""card\""}"""
2,transaction,323f6d42-a38e-4c46-83d2-6c8f3e999686,posted,Wdl ATM WES IGA BALGOWLAH HGT BALGOWL AU,-200.0,070c1d68-0ee0-477a-9679-294ea7059939,22312.06,debit,cash-withdrawal,AU00000,1bd6c08e-8457-4a74-bdd9-514df489d27e,NaN,3/08/2023,2023-08-03T00:00:00Z,NaN,"{\self\"":\""https://au-api.basiq.io/users/bca4b..."
3,transaction,1ab3a3c5-faeb-4de3-b5aa-612e5bc76fd5,posted,Non Hooli ATM Withdrawal Fee,-2.5,070c1d68-0ee0-477a-9679-294ea7059939,22512.06,debit,bank-fee,AU00000,1bd6c08e-8457-4a74-bdd9-514df489d27e,NaN,3/08/2023,2023-08-03T00:00:00Z,"{\title\"":\""\""","\""code\"":\""card\""}"""
4,transaction,d12df6a2-48b6-4f5f-a718-ccb377aa330c,posted,Transfer Platnm Homeloan 346454,-3852.5,070c1d68-0ee0-477a-9679-294ea7059939,22514.56,debit,payment,AU00000,1bd6c08e-8457-4a74-bdd9-514df489d27e,NaN,29/07/2023,2023-07-29T00:00:00Z,"{\title\"":\""Auxiliary Finance and Investment S...","\""code\"":\""641\""}"""
5,transaction,08e6f7c9-f359-48f2-b086-a55f468f2f60,posted,TFR From Transaction to CC Acc 13,-9406.0,070c1d68-0ee0-477a-9679-294ea7059939,26367.06,debit,payment,AU00000,1bd6c08e-8457-4a74-bdd9-514df489d27e,NaN,28/07/2023,2023-07-28T00:00:00Z,"{\title\"":\""Civic","Professional and Other Interest Group Services\"""
6,transaction,53570fde-c731-4e33-8224-0583650eeaa7,posted,Payroll WFRMS 15439393,17098.0,070c1d68-0ee0-477a-9679-294ea7059939,35773.06,credit,transfer,AU00000,1bd6c08e-8457-4a74-bdd9-514df489d27e,NaN,28/07/2023,2023-07-28T00:00:00Z,"{\title\"":\""Unknown\""","\""code\"":\""0\""}"""
7,transaction,55dbdd71-9976-43ec-a724-7cf620d279d4,posted,Manly Maths Tutor Wages,201.0,070c1d68-0ee0-477a-9679-294ea7059939,18675.06,credit,transfer,AU00000,1bd6c08e-8457-4a74-bdd9-514df489d27e,NaN,17/07/2023,2023-07-17T00:00:00Z,"{\title\"":\""Educational Support Services\""","\""code\"":\""822\""}"""
8,transaction,17b710a5-da3d-42f8-ae31-f9d601f95c6d,posted,MANLY WEST SCHOOL,-1422.2,070c1d68-0ee0-477a-9679-294ea7059939,18474.06,debit,payment,AU00000,1bd6c08e-8457-4a74-bdd9-514df489d27e,NaN,16/07/2023,2023-07-16T00:00:00Z,"{\title\"":\""School Education\""","\""code\"":\""802\""}"""
9,transaction,d93fb495-8a7a-4997-8a16-b3581b778471,posted,MANLY WEST SCHOOL,-51.0,070c1d68-0ee0-477a-9679-294ea7059939,19896.26,debit,payment,AU00000,1bd6c08e-8457-4a74-bdd9-514df489d27e,NaN,16/07/2023,2023-07-16T00:00:00Z,"{\title\"":\""School Education\""","\""code\"":\""802\""}"""
